In [2]:
import os
import openai

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.organization = os.getenv('openai.organization')

In [4]:
def get_completion(prompt, model='gpt-3.5-turbo'):
    messages = [{"role":"user", "content":prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0,

    )

    return response.choices[0].message['content']

In [5]:
{
    "gift" : False,
    "delivery_days" : 5,
    "price_value" : "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [6]:
customer_review = """\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been using it every other morning \
to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers out there, \
but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [8]:
prompt_template = ChatPromptTemplate.from_template(review_template)
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))])

In [16]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model = 'gpt-3.5-turbo')
response = chat(messages)
response.content


'{\n  "gift": false,\n  "delivery_days": 2,\n  "price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}'

In [19]:
print(type(response.content))

<class 'str'>


In [23]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

**Parse the LLM ouptput string into a Python dictionary**

    output을 python dictionary 형태로 리턴하기

In [25]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [26]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? \
                             Answer True if yes False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product\
                                      to arrive? If this information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [27]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [28]:
format_instructions = output_parser.get_format_instructions()

In [29]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as a gift for someone else?                              Answer True if yes False if not or unknown.\n\t"delivery_days": string  // How many days did it take for the product                                      to arrive? If this information is not found,                                      output -1.\n\t"price_value": string  // Extract any                                    sentences about the value or price, and output them as a                                     comma separated Python list.\n}\n```'

In [30]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [31]:
messages[0].content


'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing. It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "`

In [32]:
response = chat(messages)
response.content

'```json\n{\n\t"gift": false,\n\t"delivery_days": "2",\n\t"price_value": "It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."\n}\n```'

In [33]:
output_dict = output_parser.parse(response.content)
print(output_dict)
print(type(output_dict))
print(output_dict.get('delivery_days'))

{'gift': False, 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}
<class 'dict'>
2
